In [72]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import re
import geopandas as gpd

In [73]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

df = pd.read_csv('KeithsAddress - Sheet1.csv')
df['full_address'] = df['address'] + 'Brooklyn, New York, NY'
df

,address,full_address
0,17 Devoe Street,"17 Devoe StreetBrooklyn, New York, NY"
1,165 Central Avenue,"165 Central AvenueBrooklyn, New York, NY"
2,198 Scholes Street,"198 Scholes StreetBrooklyn, New York, NY"
3,11 Gunther Place,"11 Gunther PlaceBrooklyn, New York, NY"
4,194 Ralph Avenue,"194 Ralph AvenueBrooklyn, New York, NY"
...,...,...
107,132A Stanhope Street,"132A Stanhope StreetBrooklyn, New York, NY"
108,268 Metropolitan Avenue,"268 Metropolitan AvenueBrooklyn, New York, NY"
109,236 Meserole Street,"236 Meserole StreetBrooklyn, New York, NY"
110,430 Albee,"430 AlbeeBrooklyn, New York, NY"


In [74]:
# df = dsf[0]
# df.columns = df.iloc[0]
# df = df.drop(df.index[0])
# ### This next line gets ride of the Nan ###
# df = df.iloc[: , 1:]
# df = df.replace(np.nan, 'Not Available', regex=True)
# df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
#                         'Price /\rSq. Ft.':'price_per_sqft'})
# df['geo_address'] = df['building_address_city']
# df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
# df = df.replace('\n',' ',regex=True)
# df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

In [75]:
# result = pd.merge(df,df2,left_index=True, right_index=True)
# df = result

In [76]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [77]:
df['loc'] = df['full_address'].apply(geolocator.geocode, timeout=10)

In [78]:
df['loc']

0      (17 Devoe St, Brooklyn, NY 11211, USA, (40.713...
1      (165 Central Ave, Brooklyn, NY 11221, USA, (40...
2      (198 Scholes St, Brooklyn, NY 11206, USA, (40....
3      (11 Gunther Pl, Brooklyn, NY 11233, USA, (40.6...
4      (194 Ralph Ave, Brooklyn, NY 11233, USA, (40.6...
                             ...                        
107    (132A Stanhope St, Brooklyn, NY 11221, USA, (4...
108    (268 Metropolitan Ave, Brooklyn, NY 11211, USA...
109    (236 Meserole St, Brooklyn, NY 11206, USA, (40...
110    (430 Albee Square W, Brooklyn, NY 11201, USA, ...
111    (800 Flatbush Ave, Brooklyn, NY 11226, USA, (4...
Name: loc, Length: 112, dtype: object

In [79]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [80]:
df.columns

Index(['address', 'full_address', 'loc', 'point'], dtype='object')

In [81]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [82]:
df

,address,full_address,loc,point,lat,lon,altitude
0,17 Devoe Street,"17 Devoe StreetBrooklyn, New York, NY","(17 Devoe St, Brooklyn, NY 11211, USA, (40.713...","(40.71343160000001, -73.9508164, 0.0)",40.713432,-73.950816,0.0
1,165 Central Avenue,"165 Central AvenueBrooklyn, New York, NY","(165 Central Ave, Brooklyn, NY 11221, USA, (40...","(40.6989944, -73.9263607, 0.0)",40.698994,-73.926361,0.0
2,198 Scholes Street,"198 Scholes StreetBrooklyn, New York, NY","(198 Scholes St, Brooklyn, NY 11206, USA, (40....","(40.7088581, -73.9410599, 0.0)",40.708858,-73.941060,0.0
3,11 Gunther Place,"11 Gunther PlaceBrooklyn, New York, NY","(11 Gunther Pl, Brooklyn, NY 11233, USA, (40.6...","(40.6771933, -73.9115767, 0.0)",40.677193,-73.911577,0.0
4,194 Ralph Avenue,"194 Ralph AvenueBrooklyn, New York, NY","(194 Ralph Ave, Brooklyn, NY 11233, USA, (40.6...","(40.6830296, -73.92289319999999, 0.0)",40.683030,-73.922893,0.0
...,...,...,...,...,...,...,...
107,132A Stanhope Street,"132A Stanhope StreetBrooklyn, New York, NY","(132A Stanhope St, Brooklyn, NY 11221, USA, (4...","(40.69804389999999, -73.9228885, 0.0)",40.698044,-73.922888,0.0
108,268 Metropolitan Avenue,"268 Metropolitan AvenueBrooklyn, New York, NY","(268 Metropolitan Ave, Brooklyn, NY 11211, USA...","(40.7146599, -73.9585981, 0.0)",40.714660,-73.958598,0.0
109,236 Meserole Street,"236 Meserole StreetBrooklyn, New York, NY","(236 Meserole St, Brooklyn, NY 11206, USA, (40...","(40.708294, -73.9392696, 0.0)",40.708294,-73.939270,0.0
110,430 Albee,"430 AlbeeBrooklyn, New York, NY","(430 Albee Square W, Brooklyn, NY 11201, USA, ...","(40.690867, -73.98378799999999, 0.0)",40.690867,-73.983788,0.0


In [97]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11, tiles='CartoDB positron')
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'All Year\'s Real Estate Holdings')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["address"], r["altitude"])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Address: {bold_start}{info[0]}{bold_end}{new_line}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m

In [98]:
url = ("https://raw.githubusercontent.com/python-visualization/folium/master/examples/data")
nybb = f"{url}/nybb.zip"
boros = gpd.read_file(nybb)
boros

C:\Users\aefar\AppData\Local\Programs\Python\Python38\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


,BoroCode,BoroName,Shape_Leng,Shape_Area,geometry
0,5,Staten Island,330454.175933,1.623847e+09,"MULTIPOLYGON (((970217.022 145643.332, 970227...."
1,3,Brooklyn,741227.337073,1.937810e+09,"MULTIPOLYGON (((1021176.479 151374.797, 102100..."
2,4,Queens,896875.396449,3.045079e+09,"MULTIPOLYGON (((1029606.077 156073.814, 102957..."
3,1,Manhattan,358400.912836,6.364308e+08,"MULTIPOLYGON (((981219.056 188655.316, 980940...."
4,2,Bronx,464475.145651,1.186822e+09,"MULTIPOLYGON (((1012821.806 229228.265, 101278..."


In [99]:
boros_update = boros.drop([0,2,3,4])

In [100]:
boros_update

,BoroCode,BoroName,Shape_Leng,Shape_Area,geometry
1,3,Brooklyn,741227.337073,1.937810e+09,"MULTIPOLYGON (((1021176.479 151374.797, 102100..."


In [101]:
# colors added
fill_color = ['orange']
boros_update['fill_color'] = fill_color

from folium.features import GeoJsonTooltip

folium.GeoJson(boros_update,
               style_function=lambda x: {'fillColor':'orange'}
               ).add_to(m)
m

C:\Users\aefar\AppData\Local\Programs\Python\Python38\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\aefar\AppData\Local\Programs\Python\Python38\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [102]:
m.save('index.html')